In [1]:
trait RNG {
    def nextInt: (Int, RNG)
}

Intitializing Scala interpreter ...

Spark Web UI available at http://DESKTOP-5DEVR1B.mshome.net:4042
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1643875463028)
SparkSession available as 'spark'


defined trait RNG


In [2]:
case class SimpleRNG(seed: Long) extends RNG {
  def nextInt: (Int, RNG) = {
    val newSeed = (seed * 0x5DEECE66DL + 0xBL) & 0xFFFFFFFFFFFFL
    val nextRNG = SimpleRNG(newSeed)
    val n = (newSeed >>> 16).toInt
    (n, nextRNG)
  }
}

defined class SimpleRNG


In [4]:
val rng = SimpleRNG(42)

rng: SimpleRNG = SimpleRNG(42)


In [7]:
val (n1, rng2) = rng.nextInt

n1: Int = 16159453
rng2: RNG = SimpleRNG(1059025964525)


## 6-1

In [64]:
def nonNegativeInt(rng: RNG):(Int,RNG) = {
    val(n1,rng2) = rng.nextInt
    (if(n1 <0) -(n1+1) else n1,rng2)
}

nonNegativeInt: (rng: RNG)(Int, RNG)


In [65]:
nonNegativeInt(SimpleRNG(4))

res42: (Int, RNG) = (1538995,SimpleRNG(100859615679))


## 6-2

In [66]:
if(n1 < 0) -(n1+1) else n1

res43: Int = 16159453


In [67]:
def double(rng:RNG): (Double, RNG) = {
    val (n1, rng2) = nonNegativeInt(rng)
    (n1.toDouble/Int.MaxValue.toDouble,rng2)
}

double: (rng: RNG)(Double, RNG)


In [71]:
double(SimpleRNG(10))

res47: (Double, RNG) = (0.0017916266814766576,SimpleRNG(252149039181))


## 6-3

In [72]:
def intDouble(rng:RNG): ((Int,Double), RNG) = {
    val (n2, rng2) = nonNegativeInt(rng)
    val (n3, rng3) = double(rng2)
    ((n2,n3), rng3)
}

intDouble: (rng: RNG)((Int, Double), RNG)


In [73]:
def doubleInt(rng:RNG): ((Double,Int), RNG) = {
    val (n2, rng2) = double(rng)
    val (n3, rng3) = nonNegativeInt(rng2)
    ((n2,n3), rng3)
}

doubleInt: (rng: RNG)((Double, Int), RNG)


In [74]:
def double3(rng:RNG): ((Double,Double,Double),RNG) = {
    val (n2, rng2) = double(rng)
    val (n3, rng3) = double(rng2)
    val (n4, rng4) = double(rng3)
    ((n2,n3,n4),rng4)
}

double3: (rng: RNG)((Double, Double, Double), RNG)


In [75]:
type Rand[+A] = RNG => (A,RNG)

defined type alias Rand


In [77]:
val int: Rand[Int] = _.nextInt
def unit[A](a: A): Rand[A] = rng => (a, rng)
def map[A, B](s: Rand[A])(f: A => B): Rand[B] = {
    rng => {
      val (a, rng2) = s(rng)
      (f(a), rng2)
    }
  }

int: Rand[Int] = $Lambda$2383/0x00000008018c9040@3c3d70bb
unit: [A](a: A)Rand[A]
map: [A, B](s: Rand[A])(f: A => B)Rand[B]


In [79]:
def nonNegativeEven:Rand[Int] = {
    map(nonNegativeInt)(i => i - i % 2)
}

nonNegativeEven: Rand[Int]


원래는 def foo(param): returnType 을 넣어주는게 일반적인 형태인데
Rand 자체를 type으로 등록해서 RNG => (A,RNG)로 표현이 되므로 위와 같이 쓰면
nonNegativeEven(rng:RNG):(A,RNG) 와 같은 뜻임

In [81]:
nonNegativeEven(SimpleRNG(10))

res49: (Int, RNG) = (3847488,SimpleRNG(252149039181))


In [82]:
def double:Rand[Double] = {
    map(nonNegativeInt)(i => i/Int.MaxValue.toDouble)
}

double: Rand[Double]
